# Sistema de EDO de Rössler

En 1970, el químico biológico alemán [Otto Rössler](https://en.wikipedia.org/wiki/Otto_R%C3%B6ssler) 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/Otto_E._R%C3%B6ssler_at_transmediale.08.jpg/234px-Otto_E._R%C3%B6ssler_at_transmediale.08.jpg)

introduce el siguiente sistema de tres ecuaciones diferenciales ordinarias acopladas:

\begin{align}
y'_1(t) &= -y_2(t) -y_3(t) \\
y'_2(t) &= y_1(t) \, + \, \alpha \,  y_2(t)\\
y'_3(t) &=\beta\, +\, y_3(t)\, (y_1(t)\,-\,\gamma)
\end{align}

El interés de Rössler se originó en el estudio de **órbitas caóticas**.

In [ ]:
using Plots
using LaTeXStrings
using BenchmarkTools
using Random
using FileIO
using JLD2
using Dates
using Base.Threads # Para paralelizar con threads (hilos), i.e. paralelización dentro de un nodo.

In [ ]:
# Vemos cuantos threads tenemos
nthreads()

In [ ]:
function paso_rk4(f,x,t,h)
    k1 = h*f(x,t)
    k2 = h*f(x+k1/2,t+h/2)
    k3 = h*f(x+k2/2,t+h/2)
    k4 = h*f(x+k3,t+h)
    return x+(k1+2k2+2k3+k4)/6
end

In [ ]:
function integrador_EDO!(method,f,vt,vx)
    @assert size(vx)[1]>1
    for s in 2:length(vt)
        h = vt[s]-vt[s-1]
        x = view(vx,:,s-1)
        dx = view(vx,:,s)
        method!(f!,dx,x,vt[s],h)
    end
    return vx
end

In [ ]:
function integrador_EDO!(paso,f,vt,vx)
    for i in 2:length(vt)
        vx[:,i] = paso(f,vx[:,i-1],vt[i-1],vt[i]-vt[i-1])         
    end
    return vx
end

In [ ]:
function simulador(integrador!,num_muestras,parametros,vt)
    vx = zeros(3,length(vt))
    muestras = []    
    for p in parametros
        f_rossler(x,t) = [-x[2]-x[3],x[1]+p[1]*x[2],p[2]+x[3]*(x[1]-p[3])]
        #for s in 1:num_muestras        
        @threads for s in 1:num_muestras        
            # seteamos la condición inicial
            rand!(view(vx,:,1)) 
            vx[:,1] .*= 1e-1
            # integramos
            muestra = integrador_EDO!(paso_rk4,f_rossler,vt,vx)
            # guardamos la muestra
            push!(muestras,[p,vt,copy(muestra)]) # copiamos la muestra para grabarla
        end
    end
    return muestras
end

In [ ]:
datos = Dict()
try
    # Usamos un diccionario pre-existente si existe
    global datos = load("simulador-datos.jld2")
catch
    # sinó, usamos el vacío
    println("Usando un diccionario vacío!")
end

In [ ]:
# Muestreamos 100 trayectorias para dos elecciones de los valores de los parametros.
muestras = simulador(integrador_EDO!,1000,[(0.2,0.2,5.7) (0.1,0.1,14.0)],0.0:0.1:500.0)

In [ ]:
# Guardamos los resultados de la nueva simulación usando la fecha y la hora como "llave"
datos["simulador-muestras-"*Dates.format(Dates.now(),"yyyy-mm-dd-HH-MM-SS")]=muestras

In [ ]:
# Guardamos la nueva versión del diccionario para que los datos puedan ser analizados en otras notebooks.
save("simulador-datos.jld2",datos)

In [ ]:
(a,b,c),vt,vx=muestras[2]

In [ ]:
plot()
plot!(xlabel=L"t")
plot!(vt,vx[1,:],label=L"x(t)")
plot!(vt,vx[2,:],label=L"y(t)")
plot!(vt,vx[3,:],label=L"z(t)")